# hurricane-net
[https://github.com/hammad93/hurricane-net](https://github.com/hammad93/hurricane-net)

![Hurricane Irma 2017](img/irma.jpg "Maximum sustained winds of 185mph")

## Train
LSTM Window

## Test